# Running an Individual Test Suite

This notebook shows how to run an individual test suite that is part of the ValidMind Developer Framework.  These instructions include the code required to:

- Load the demo dataset
- List available test suites
- Run the data validation test suite
- Run the model validation test suite
- Prepocess the raw dataset
- Train a model for testing
- Import and run the individual test suite
- Initialize ValidMind objects
- Run the binary classification test suite

## Before you begin

::: {.callout-tip}
### New to ValidMind? 
For access to all features available in this notebook, create a free ValidMind account. 

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

In [ ]:
%pip install -q validmind

## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook. As you will see later, documentation projects are useful because they act as containers for model documentation and validation reports and they enable you to organize all of your documentation work in one place. 

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Customer Churn Model`** and **`Initial Validation`** for the model name and type, give the project a unique  name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [ ]:
## Replace with code snippet from your documentation project ##

import validmind as vm

vm.init(
  api_host = "https://api.prod.validmind.ai/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)

In [ ]:
%matplotlib inline

import xgboost as xgb

## Load the Demo Dataset

In [ ]:
# You can also import taiwan_credit like this:
# from validmind.datasets.classification import taiwan_credit as demo_dataset
from validmind.datasets.classification import customer_churn as demo_dataset

df = demo_dataset.load_data()

In [ ]:
vm_dataset = vm.init_dataset(
    input_id="raw_dataset",
    dataset=df,
    target_column=demo_dataset.target_column,
    class_labels=demo_dataset.class_labels
)

## List available test suites

In [ ]:
vm.test_suites.list_suites()

## Run the data validation test suite

In [ ]:
tabular_suite = vm.run_test_suite("tabular_dataset", dataset=vm_dataset)

## Run the model validation test suite

We will need to preprocess the dataset and produce the training, test and validation splits first.

### Prepocess the raw dataset

In [ ]:
train_df, validation_df, test_df = demo_dataset.preprocess(df)

### Train a model for testing

We train a simple customer churn model for our test.

In [ ]:
x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

## Import and run the individual test suite

### Initialize ValidMind objects

We initize the objects required to run test suites using the ValidMind framework.

In [ ]:
vm_train_ds = vm.init_dataset(
    input_id="train_dataset",
    dataset=train_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    input_id="test_dataset",
    dataset=test_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_model = vm.init_model(
    model,
    input_id="model"
)

### Assign predictions to the datasets
We can now use the `assign_predictions()` method from the `Dataset` object to link existing predictions to any model:

In [ ]:
vm_train_ds.assign_predictions(
    model=vm_model,
    prediction_values=list(model.predict(x_train))
)
vm_test_ds.assign_predictions(
    model=vm_model,
    prediction_values=list(model.predict(x_val))
)

### Run the binary classification test suite

In [ ]:
model_suite = vm.run_test_suite(
    "classifier_model_validation",
    inputs = {
        "dataset": vm_test_ds,
        "datasets": (vm_train_ds, vm_test_ds),
        "model":vm_model,
        "models":[vm_model]
    }
)